In [1]:
import numpy as np

# Defining global variables

In [8]:
#For testing purposes

KEY = ''
MESSAGE = ''

![title](img/sbox.png)

In [7]:
sbox1 = [['101', '010', '001', '110', '011', '100', '111', '000'],
         ['001', '100', '110', '010', '000', '111', '101', '011']]

sbox2 = [['100', '000', '110', '101', '111', '001', '011', '010'],
         ['101', '011', '000', '111', '110', '010', '001', '100']]

Helper functions

In [27]:
def xOR(Y, K):
    res = ''
    n = len(Y)
    for i in range(n):
        res += str(int(Y[i]) ^ int(K[i]))

    return res

# Round functions

![ith Round of DES](img/round.png)

## Expand function

![The Expander Function](img/expander_func.png)

In [18]:
def expander(X):
    mapdict = {1: 1, 2: 2, 3: 4, 4: 3, 5: 4, 6: 3, 7: 5, 8: 6}
    expandedX = ['0'] * 8

    for i in mapdict.keys():
        expandedX[i - 1] = X[mapdict[i] - 1]

    return "".join(expandedX)

## Substitution using $S_1$ and $S_2$

In [25]:
def substitution(X):
    X1 = X[:4]
    X2 = X[4:]

    Sub1 = sbox1[int(X1[0])][int(X1[1:], 2)]
    Sub2 = sbox2[int(X2[0])][int(X2[1:], 2)]

    return Sub1 + Sub2

## $f(R_{i-1},K_i)$

In [51]:
def f(R, K):

    try:
        assert (len(R) == 6)
    except:
        raise (AssertionError("Length of Ri is not 6 bits!"))

    try:
        assert (len(K) == 8)
    except:
        raise (AssertionError("Length of Ki is not 8 bits!"))

    expandedR = expander(R)
    XORed = xOR(expandedR, K)
    return substitution(XORed)

## Round keys

In [87]:
def roundkey(K, i):

    K2 = K[0:i - 1]
    K1 = K[i - 1:]

    return (K1 + K2)[:-1]

In [119]:
def inv_roundkey(K, i, rounds):

    K2 = K[0:rounds - i]
    K1 = K[rounds - i:]

    return (K1 + K2)[:-1]

# Implementing DES

!['One Round of a Feistel System'](img/ith-round.png)

## Encryption

In [146]:
def encrypt(message, key, rounds=4):
    try:
        assert (len(message) == 12)
    except:
        raise (AssertionError("Input message length should be 12 bits!"))

    try:
        assert (len(key) == 9)
    except:
        raise (AssertionError("Key length should be 9 bits!"))

    out = message

    for i in range(1, rounds + 1):
        L_prev = out[:6]
        R_prev = out[6:]

        Ki = roundkey(key, i)
        print("L" + str(i - 1) + " :", L_prev)
        print("R" + str(i - 1) + " :", R_prev)
        print("K" + str(i) + " :", Ki)
        print()
        out = R_prev + xOR(L_prev, f(R_prev, Ki))

    print("L" + str(i) + " :", out[:6])
    print("R" + str(i) + " :", out[6:])

    return out[6:] + out[:6]

In [147]:
encrypt('100010110101','111000111',rounds=2)

L0 : 100010
R0 : 110101
K1 : 11100011

L1 : 110101
R1 : 001010
K2 : 11000111

L2 : 001010
R2 : 001101


'001101001010'

## Decryption

In [140]:
def decrypt(cipher, key, rounds=4):
    try:
        assert (len(cipher) == 12)
    except:
        raise (AssertionError("Input ciphertext length should be 12 bits!"))

    try:
        assert (len(key) == 9)
    except:
        raise (AssertionError("Key length should be 9 bits!"))

    out = cipher

    for i in range(1, rounds + 1):
        L_prev = out[:6]
        R_prev = out[6:]

        Ki = inv_roundkey(key, i, rounds)
        print("L" + str(i - 1) + " :", L_prev)
        print("R" + str(i - 1) + " :", R_prev)
        print("K" + str(rounds - i + 1) + " :", Ki)
        print()
        out = R_prev + xOR(L_prev, f(R_prev, Ki))

    print("L" + str(i) + " :", out[:6])
    print("R" + str(i) + " :", out[6:])
    return out[6:] + out[:6]

In [141]:
decrypt('001101001010','111000111',rounds=2)

L0 : 001101
R0 : 001010
K2 : 11000111

L1 : 001010
R1 : 110101
K1 : 11100011

L2 : 110101
R2 : 100010


'100010110101'